In [8]:
import os
import glob
import pandas as pd

In [3]:
path_results = "/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_endodiff/debug_May2021/REVISION/CRM_interaction_chr22/results/"

In [4]:
fname = os.path.join(path_results,"*.tsv")
files = glob.glob(fname)
print (files)

['/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_endodiff/debug_May2021/REVISION/CRM_interaction_chr22/results/ENSG00000100206_10.tsv', '/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_endodiff/debug_May2021/REVISION/CRM_interaction_chr22/results/ENSG00000100206_290.tsv', '/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_endodiff/debug_May2021/REVISION/CRM_interaction_chr22/results/ENSG00000100206_0.tsv']


In [6]:
file = files[0]
file

'/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_endodiff/debug_May2021/REVISION/CRM_interaction_chr22/results/ENSG00000100206_10.tsv'

In [9]:
df = pd.read_csv(file, index_col=0)
df.head()

,chrom,pv,variant
0,22,0.472747,22_38818243_G_A
1,22,0.991658,22_38818676_C_G
2,22,0.991658,22_38819194_G_C
3,22,0.433430,22_38819613_G_A
4,22,0.457743,22_38821504_G_A


In [10]:
nsnps = int(len(df))
nsnps

10

In [18]:
line = str(file).split("/")
# line

In [19]:
gene = str(line[-1]).split("_")[0]
gene

'ENSG00000100206'

In [20]:
chunk = str(line[-1]).split("_")[1].split(".")[0]
chunk

'10'

In [21]:
chrom = df['chrom'].values[0]
chrom

22

In [22]:
pval = df['pv'].values
pval

array([0.47274666, 0.9916579 , 0.9916579 , 0.43343022, 0.45774327,
       0.97073514, 0.53811297, 0.59874603, 0.23222526, 0.45774327])

In [23]:
pval[pd.isnull(pval)]=1
pval

array([0.47274666, 0.9916579 , 0.9916579 , 0.43343022, 0.45774327,
       0.97073514, 0.53811297, 0.59874603, 0.23222526, 0.45774327])

In [ ]:
import sys
import re
import glob
import os
import h5py
import pdb
import pandas as pd
import scipy as sp
import numpy as np
import statsmodels
# import limix.stats.fdr as fdr

def smartAppend(table,name,value):
    """ helper function for appending in a dictionary """
    if name not in table.keys():
        table[name] = []
    table[name].append(value)

def dumpDictHdf5(RV,o):
    """ Dump a dictionary where each page is a list or an array """
    for key in RV.keys():
        o.create_dataset(name=key,data=sp.array(RV[key]),chunks=True,compression='gzip')

def smartDumpDictHdf5(RV,o):
    """ Dump a dictionary where each page is a list or an array or still a dictionary (in this case, it iterates) """
    for key in RV.keys():
        if type(RV[key])==dict:
            g = o.create_group(key)
            smartDumpDictHdf5(RV[key],g)
        else:
            o.create_dataset(name=key,data=sp.array(RV[key]),chunks=True,compression='gzip')

def fdr(p_vals):

    from scipy.stats import rankdata
    ranked_p_values = rankdata(p_vals)
    fdr = p_vals * len(p_vals) / ranked_p_values
    fdr[fdr > 1] = 1

    return fdr

path_results = "/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_endodiff/debug_May2021/REVISION/CRM_interaction_chr22/results/"


if __name__ == '__main__':

    fname = os.path.join(path_results,"*.tsv")
    #prinm(outfilename)
    files = glob.glob(fname)
    # print (files)
    #import pdb; pdb.set_trace()
    x = 0
    table = {}
    count_success = 0
    count_failed = 0
    #breakpoint()
    for file in files:
        #breakpoint()
        # if re.search("perm",file) is not None:
        #     continue
        # x += 1
        if x%500 == 0: print (x)
        df = pd.read_csv(file, index_col=0)
        nsnps = int(len(df))
        if nsnps==0:
            continue
        line = str(file).split("/")
        gene = str(line[-1]).split(".")[0]
        chrom = df['chrom'].values[0]
        #print(gene)
        pval = df['pv'].values
        #pval[np.isnan(pval)]=1
        pval[pd.isnull(pval)]=1
        for i in range(nsnps):
            try:
                #import pdb; pdb.set_trace()
                temp={}
                temp['gene'] = gene
                temp['n_snps'] = nsnps
                temp['chrom'] = chrom
                #print(nsnps)
                temp['pv_raw'] = df['pv'].values[i]
                temp['snpID'] = df['variant'].values[i]
                #FWER adjusted (gene-level) pvalue
                temp['pv'] = nsnps*temp['pv_raw']
                if temp['pv']>1: temp['pv'] = 1
                if temp['pv']<0: temp['pv'] = 0
                count_success+=1
            
            except:
                count_failed+=1
                continue
        
            for key in temp.keys():
                smartAppend(table,key,temp[key])
    
    #import pdb; pdb.set_trace()
    for key in table.keys():
        table[key] = sp.array(table[key])
    print (count_success)
    
    df = pd.DataFrame.from_dict(table)
    import os
    outfile = "summary.csv"
    myp = os.path.join(path_results,outfile)
    df.to_csv(myp)